# Using OpenAI's Function Calling

Anton Antonov   
[RakuForPrediction at WordPress](https://rakuforprediction.wordpress.com)   
May 2025

-----

## Introduction

This notebook shows how to do [Function Calling](https://platform.openai.com/docs/guides/function-calling) workflows with Large Language Models (LLMs) of OpenAI. 

(The Raku package ["WWW::OpenAI"](https://github.com/antononcube/Raku-WWW-OpenAI) is used.)

-----

## Setup

Load packages:

In [1]:
use WWW::OpenAI;
use JSON::Fast;

Choose a model:

In [ ]:
my $model = "gpt-4.1";

gpt-4.1

------

## Workflow

### Define a local function

This is the "tool" to be communicated to OpenAI.

In [ ]:
# Define the local function
sub get_current_weather(Str $location, Str $unit = "fahrenheit") returns Str {
    return "It is currently sunny in $location with a temperature of 72 degrees $unit.";
}

# Define the function specification
my $function-spec = {
    type => "function",
    function => {
        name => "get_current_weather",
        description => "Get the current weather for a given location",
        parameters => {
            type => "object",
            properties => {
                location => {
                    type => "string",
                    description => "The city and state, e.g., San Francisco, CA"
                },
                unit => {
                    type => "string",
                    enum => ["celsius", "fahrenheit"],
                    description => "The temperature unit to use"
                }
            },
            required => ["location"]
        }
    }
};

{function => {description => Get the current weather for a given location, name => get_current_weather, parameters => {properties => {location => {description => The city and state, e.g., San Francisco, CA, type => string}, unit => {description => The temperature unit to use, enum => [celsius fahrenheit], type => string}}, required => [location], type => object}}, type => function}

### First communication with OpenAI

In [ ]:
# Initialize messages and tools
my @messages =
    {role => "system", content =>  "You are a helpful assistant that can provide weather information."},
    {role => "user", content => "What's the weather in Boston, MA?"}
    ;

my @tools = [$function-spec,];

# Send the first chat completion request
my $response = openai-chat-completion(
    @messages,
    #functions => @tools,
    :@tools,
    #tool_choice => "auto",
    :$model,
    max-tokens => 4096,
    format => "raku"
);

[{finish_reason => tool_calls, index => 0, logprobs => (Any), message => {annotations => [], content => (Any), refusal => (Any), role => assistant, tool_calls => [{function => {arguments => {"location":"Boston, MA"}, name => get_current_weather}, id => call_vce1qmDSfHovNoN5yUJotp5s, type => function}]}}]

### Refine the response with functional calls

In [ ]:
# Process the response
my @messages2 = @messages;

my $assistant-message = $response[0]<message>;
if $assistant-message<tool_calls> {

    @messages2.push: {
        role => "assistant",
        tool_calls => $assistant-message<tool_calls>
    };


    my $tool-call = $assistant-message<tool_calls>[0];
    my $function-name = $tool-call<function><name>;
    my $function-args = from-json($tool-call<function><arguments>);
    
    if $function-name eq "get_current_weather" {
        my $result = get_current_weather(
            $function-args<location>,
            $function-args<unit> // "fahrenheit"
        );
        @messages2.push: {
            role => "tool",
            content => $result,
            tool_call_id => $tool-call<id>
        };
        # Send the second request with function result
        my $final-response = openai-chat-completion(
            @messages2,
            :@tools,
            #tool_choice => "auto",
            :$model,
            format => "raku"
        );
        say "Assistant: $final-response[0]<message><content>";
    }
} else {
    say "Assistant: $assistant-message<content>";
}

Assistant: The current weather in Boston, MA is sunny with a temperature of 72
